# Differentiating CVQNN layers with Piquasso and Tensorflow

In Piquasso, one can easily create Continuous-Variable Quantum Neural Networks (CVQNN) circuits using the [Piquasso CVQNN](https://docs.piquasso.com/misc/cvqnn.html) module. In these circuits, the layers are constructed according to [Continuous-variable quantum neural networks](https://arxiv.org/abs/1806.06871).

In [50]:
import piquasso as pq

d = 2  # Number of qumodes

layer_count = 10  # Number of CVQNN layers

# Generating random weights
weights = pq.cvqnn.generate_random_cvqnn_weights(layer_count=layer_count, d=d)

# Creating CVQNN layers as a Piquasso subprogram
cvqnn_layers = pq.cvqnn.create_layers(weights)

Piquasso automatically sets up a subprogram containing the instructions of the desired CVQNN layer. Now we can embed this subprogram in any Piquasso program. Let's choose the input state as a pure displaced state.

In [51]:
# The program definition
with pq.Program() as program:
    pq.Q() | pq.Vacuum()

    for i in range(d):
        pq.Q(i) | pq.Displacement(r=0.1)

    pq.Q() | cvqnn_layers

Now, we need to choose the simulator which executes the instructions. Since a CVQNN layer includes non-linear terms, we definitely need to perform the simulation in Fock space. Since our initial state is pure, we can use [PureFockSimulator](https://docs.piquasso.com/simulators/fock.html#module-piquasso._backends.fock.pure.simulator).

In [52]:
cutoff = 5

simulator = pq.PureFockSimulator(d, pq.Config(cutoff=cutoff))

final_state = simulator.execute(program).state

After obtaining the state, we can calculate several things, e.g. the expectation value of the position operator on mode 0.

In [53]:
print("Mean position on mode 0:")
final_state.mean_position(mode=0)

Mean position on mode 0:


0.13816424588589984

In order to differentiate quantitities, we need to modify the simulation. In itself, `PureFockSimulator` is unable to perform automatic differentiation. In order to do that, we can use `TensorflowCalculator`, which replaces NumPy to TensorFlow under the hood. For a concrete example, let the loss function be
$$
J(w) = || \ket{\psi(w)} - \ket{\psi_*} ||_2,
$$
where $\ket{\psi(w)}$ is the final state of the circuit, and $\ket{\psi_*}$ is some random final state.

In [54]:
import numpy as np
from scipy.special import comb

state_vector_size = comb(d + cutoff - 1, cutoff - 1, exact=True)

psi_star = np.random.rand(state_vector_size) + 1j * np.random.rand(state_vector_size)

psi_star /= np.sum(np.abs(psi_star))

Then, by using [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape), we can differentiate this loss function.

In [55]:
import tensorflow as tf

tf.get_logger().setLevel("ERROR")  # Turns off complex->float casting warnings

simulator = pq.PureFockSimulator(
    d, pq.Config(cutoff=cutoff), calculator=pq.TensorflowCalculator()
)

w = tf.Variable(weights)
psi_star = tf.Variable(psi_star)

with tf.GradientTape() as tape:
    cvqnn_layers = pq.cvqnn.create_layers(w)

    with pq.Program() as program:
        pq.Q() | pq.Vacuum()

        for i in range(d):
            pq.Q(i) | pq.Displacement(r=0.1)

        pq.Q() | cvqnn_layers

    simulator.execute(program)

    final_state = simulator.execute(program).state

    psi = final_state.state_vector

    loss = tf.math.reduce_sum(tf.math.abs(psi - psi_star))

loss_grad = tape.gradient(loss, w)

print(f"Loss: {loss.numpy()}")
print(f"Loss gradient: {loss_grad.numpy()}")

Loss: 1.9143935803818333
Loss gradient: [[ 2.74908233e-02  1.90579159e-02 -1.36359733e-01 -7.89900742e-01
   6.31235971e-02  4.51359355e-02  1.37501359e-02 -1.49146064e-01
   7.07124282e-01  6.13503400e-01 -1.86550761e-02 -3.30165378e-03
  -1.93856132e-01 -8.64247494e-02]
 [-2.56806492e-02 -1.09311576e-02 -1.56869982e-01 -1.04084859e+00
   2.62702500e-02 -2.82813720e-02 -2.47865528e-04 -1.62468196e-01
   7.93002340e-01  6.26729485e-01  3.06385925e-03  6.08110349e-03
  -1.93358322e-01 -1.24251943e-01]
 [-9.50446575e-04  1.33989483e-02 -1.72803285e-01 -9.41667791e-01
   4.26243569e-02 -1.48916159e-02 -1.07390412e-02 -1.76735043e-01
   7.95963200e-01  6.87943272e-01 -2.03485134e-02  3.07257792e-04
  -2.62790306e-01 -1.16476344e-01]
 [-3.72450937e-02 -2.08769879e-02 -1.76206568e-01 -1.02481323e+00
   1.31557643e-02  4.54583859e-02 -3.87547982e-02 -1.36321748e-01
   5.59320908e-01  7.49140192e-01  2.20169407e-02  9.05257291e-03
  -1.29899478e-01 -1.74896858e-01]
 [ 1.05019969e-04  2.9689869

However, Piquasso is written in a way that it supports `tf.function` (see [Better performance with tf.function](https://www.tensorflow.org/guide/function)) one can also use `tf.function` for this task. Refactoring everything into a function, we can use the `tf.function` decorator. Note, that we have to avoid side effects in any function decorated with `tf.function`, because side effects are only executed at the tracing step. Therefore, instantiation of `pq.Program` should happen by providing the instructions as constructor arguments, instead of using the `with` keyword.

In [56]:
def calculate_loss(w, psi_star, cutoff):
    d = pq.cvqnn.get_number_of_modes(w.shape[1])

    simulator = pq.PureFockSimulator(
        d,
        pq.Config(cutoff=cutoff, normalize=False),
        calculator=pq.TensorflowCalculator(decorate_with=tf.function),
    )

    with tf.GradientTape() as tape:
        cvqnn_layers = pq.cvqnn.create_layers(w)

        preparation = [pq.Vacuum()]

        program = pq.Program(instructions=preparation + cvqnn_layers.instructions)

        simulator.execute(program)

        final_state = simulator.execute(program).state

        psi = final_state.state_vector

        loss = tf.math.reduce_sum(tf.math.abs(psi - psi_star))

    return loss, tape.gradient(loss, w)


improved_calculate_loss = tf.function(calculate_loss)

loss, loss_grad = improved_calculate_loss(w, psi_star, cutoff)

print(f"Loss: {loss.numpy()}")
print(f"Loss gradient: {loss_grad.numpy()}")

Loss: 1.894765747849647
Loss gradient: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.67345017e-01
  -2.54874175e-02  3.58265820e-03  5.05677534e-04  1.94302250e-04
  -3.64319186e-01 -3.45216700e-01 -1.69090137e-02 -3.40747922e-03
  -1.60837509e-02 -1.98875566e-02]
 [-5.86457902e-03 -1.21035878e-03 -1.55043526e-02 -8.28271323e-01
  -5.31066904e-02 -4.84554544e-03 -5.80812625e-05 -1.76544370e-02
  -2.08216692e-01 -3.76865336e-01  3.10981158e-03  5.96160077e-03
  -1.33899841e-02 -4.03384195e-02]
 [-1.13481850e-04  2.70243642e-03 -1.72470618e-02 -7.65970527e-01
  -5.66513626e-02 -6.72089071e-03 -2.60329493e-03 -2.47237259e-02
  -2.00357768e-01 -2.97848746e-01 -2.06781753e-02  3.50210960e-04
  -5.56693537e-02 -3.67677830e-02]
 [-2.27468448e-02 -5.52633668e-03 -3.98755645e-02 -8.25417102e-01
  -5.93043926e-02 -1.33327910e-03 -1.01042051e-02 -2.91571317e-02
  -2.15611339e-01 -4.28961878e-01  2.08739184e-02  8.76651319e-03
   1.37371206e-03 -3.89518705e-02]
 [-1.87892157e-03  5.37833154

The first run is called the tracing step, and it takes some time, because Tensorflow captures a [tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph) here. The size of the graph can be decreased by passing the `decorate_with=tf.function` argument to `pq.TensorflowCalculator`, which also decreases the execution time of the tracing step. After the first run, a significant speedup is observed. We can also compare the runtimes of the compiled and non-compiled function.

In [57]:
import time
import numpy as np

regular_runtimes = []
improved_runtimes = []

for i in range(10):
    w = tf.Variable(pq.cvqnn.generate_random_cvqnn_weights(layer_count, d))

    start_time = time.time()
    calculate_loss(w, psi_star, cutoff)
    end_time = time.time()

    regular_runtimes.append(end_time - start_time)

    start_time = time.time()
    improved_calculate_loss(w, psi_star, cutoff)
    end_time = time.time()

    improved_runtimes.append(end_time - start_time)

print(f"Regular: {np.mean(regular_runtimes)} s (+/- {np.std(regular_runtimes)} s).")
print(f"Improved: {np.mean(improved_runtimes)} s (+/- {np.std(improved_runtimes)} s).")

Regular: 3.4202134370803834 s (+/- 0.5104111165502103 s).
Improved: 0.0135833740234375 s (+/- 0.0009120222624867234 s).


But that is not everything yet! One can also create a similar function with the `jit_compile=True` flag, since every operation in Piquasso can be JIT-compiled using XLA through `tf.function`.

In [58]:
jit_compiled_calculate_loss = tf.function(jit_compile=True)(calculate_loss)

jit_compiled_calculate_loss(w, psi_star, cutoff)

2024-02-26 11:31:14.942992: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 5m4.497154802s

********************************
[Compiling module a_inference_calculate_loss_3303349__XlaMustCompile_true_config_proto_3175580994766145631_executor_type_11160318154034397263_.80211] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


(<tf.Tensor: shape=(), dtype=float64, numpy=1.861646546992796>,
 <tf.Tensor: shape=(10, 14), dtype=float64, numpy=
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -3.79367684e-01,  6.09711604e-01,  2.45689046e-04,
         -2.38614997e-04, -1.62932343e-03, -5.77995886e-01,
         -6.19436366e-01,  6.32017395e-03,  7.60270405e-04,
          3.31421985e-03, -7.19956868e-04],
        [-2.92014789e-03,  1.86369322e-03,  2.82715730e-03,
         -3.29063648e-01,  6.37470378e-01, -1.38643771e-02,
         -1.89973120e-03,  1.88512334e-02, -7.47652945e-01,
         -4.52539548e-01, -2.44055629e-03,  3.16780214e-03,
          3.28290996e-02, -2.80871126e-03],
        [-9.76887143e-03,  2.62403637e-04,  1.61482734e-02,
         -2.14483059e-01,  6.41185058e-01, -2.89731669e-03,
         -1.29745999e-03,  1.56235068e-02, -8.20582857e-01,
         -4.15654874e-01,  3.22268644e-04, -1.91732936e-02,
          3.20189738e-02, -4.42354155e-02],
        [ 1.28716325e-02,  4.0600

Compiling the same function takes significantly more time, but the compilation step results in an extra order of magnitude runtime improvement:

In [59]:
jit_compiled_runtimes = []

for i in range(10):
    w = tf.Variable(pq.cvqnn.generate_random_cvqnn_weights(layer_count, d))

    start_time = time.time()
    jit_compiled_calculate_loss(w, psi_star, cutoff)
    end_time = time.time()

    jit_compiled_runtimes.append(end_time - start_time)

print(f"Regular:\t{np.mean(regular_runtimes)} s (+/- {np.std(regular_runtimes)} s).")
print(f"Improved:\t{np.mean(improved_runtimes)} s (+/- {np.std(improved_runtimes)} s).")
print(
    f"JIT compiled:\t{np.mean(jit_compiled_runtimes)} s "
    f"(+/- {np.std(jit_compiled_runtimes)} s)."
)

Regular:	3.4202134370803834 s (+/- 0.5104111165502103 s).
Improved:	0.0135833740234375 s (+/- 0.0009120222624867234 s).
JIT compiled:	0.0016646862030029296 s (+/- 0.0001866165683130647 s).


We are able to use this function for. e.g., quantum state learning. Consider the state
$$
\ket{\psi_*} = \frac{1}{\sqrt{2}} \left ( \ket{03} + \ket{30} \right ),
$$
which is the so-called NOON state for $N=3$. We can produce this using Piquasso:

In [60]:
with pq.Program() as target_state_preparation:
    pq.Q(all) | pq.StateVector([0, 3]) / np.sqrt(2)
    pq.Q(all) | pq.StateVector([3, 0]) / np.sqrt(2)


target_state = simulator.execute(target_state_preparation).state

target_state_vector = target_state.state_vector

psi_star = tf.Variable(target_state_vector)

Now we can demonstrate the speed of the JIT-compiled calculation by creating a simple optimization algorithm as follows:

In [61]:
learning_rate = 0.01
iterations = 10000

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

w = tf.Variable(pq.cvqnn.generate_random_cvqnn_weights(layer_count, d))


for i in range(iterations):
    loss, loss_grad = jit_compiled_calculate_loss(w, psi_star, cutoff)

    optimizer.apply_gradients(zip([loss_grad], [w]))

    if (i + 1) % (iterations // 20) == 0:
        print(f"{i+1}:\t\t", loss)

print("Final loss:\t", loss)

500:		 tf.Tensor(0.8148386265792411, shape=(), dtype=float64)
1000:		 tf.Tensor(0.7873113596246494, shape=(), dtype=float64)
1500:		 tf.Tensor(0.760595638101206, shape=(), dtype=float64)
2000:		 tf.Tensor(0.8061251242374848, shape=(), dtype=float64)
2500:		 tf.Tensor(0.8089312093720207, shape=(), dtype=float64)
3000:		 tf.Tensor(0.37133245801501613, shape=(), dtype=float64)
3500:		 tf.Tensor(0.276721172472077, shape=(), dtype=float64)
4000:		 tf.Tensor(0.27085156102162544, shape=(), dtype=float64)
4500:		 tf.Tensor(0.25758575067817163, shape=(), dtype=float64)
5000:		 tf.Tensor(0.2605988937298275, shape=(), dtype=float64)
5500:		 tf.Tensor(0.23163654370722436, shape=(), dtype=float64)
6000:		 tf.Tensor(0.22739231353193629, shape=(), dtype=float64)
6500:		 tf.Tensor(0.2363594837005884, shape=(), dtype=float64)
7000:		 tf.Tensor(0.22506580630254558, shape=(), dtype=float64)
7500:		 tf.Tensor(0.22863825020126977, shape=(), dtype=float64)
8000:		 tf.Tensor(0.228557172932431, shape=(), dtyp

We can use the final weights to calculate the final state, and calculate its fidelity with the target state.

In [63]:
program = pq.cvqnn.create_program(w)

final_state = simulator.execute(program).state

print("Final state fidelity: ", final_state.fidelity(target_state))

Final state fidelity:  0.9992644012667061
